# Imports

In [1]:
from pathlib import Path
import sys
import warnings

import plotly.express as px
import seaborn as sns
import nbformat

import pandas as pd

abs_path = Path().resolve()
sys.path.append(str(abs_path.parent / "modules"))

import pandas as pd

from files_funcs import *
from docx_funcs import *

warnings.filterwarnings("ignore")

# Config

In [2]:
main_dir = abs_path.parent.parent

config_path = abs_path.parent.parent / "config" / "config.yaml"
config = load_yaml(config_path)

data_dir = main_dir / "data"
uc_data_dir = data_dir / "raw_data" / "HMI"
ssts_data_dir = data_dir / "raw_data" / "SSTS"

columns = (
        config["ds_topology"]["main_columns"]
        + list(set(config["ds_topology"]["hmi_subheaders"].values()))
        + config["ds_topology"]["targets"]
)
columns = [col.lower().replace(" ", "_") for col in columns]

# Loading

In [3]:
ds = create_ds(uc_data_dir, ssts_data_dir, columns)
ds

Warning! [Errno 2] No such file or directory: '/Users/beesknight/Desktop/code/bogdan/atom-compliance-ml/data/raw_data/SSTS/SSTS-26160.docx'


id                                          case_name  \
0    6583                 Driver initiate a call through SWP   
1   30371                                 ERA Self-diagnosis   
2   28561                    Setting Hotspot name & password   
3   26160              Filtering the internet radio stations   
4   25957                  Mute/unmute the FM Radio playback   
5   31523            Adding Internet Radio to Favorites List   
6   26771                     Turn on and off hotspot via VA   
7    8604                  Source selection for Media output   
8   26161                        FM Radio Stations switching   
9    8800                       Receiving Call Notifications   
10  11467  Revoke access to the vehicle from a driver or ...   
11   8692      Emergency Service Communication (ERA-Glonass)   

                                         full_uc_text  \
0   [I-6583]  Driver initiate a call through SWP\n...   
1   [I-30371]  ERA Self-diagnosis\nActors:\nERA-Gl...   
2   [I-28561]  Setting Hotspot name & password\nUs...   
3   [I-26160]  Filtering the internet radio statio...   
4   [I-25957]  Mute/unmute the FM Radio playback\n...   
5   [I-31523]  Adding Internet Radio to Favorites ...   
6   [I-26771]  Turn on and off hotspot via VA\n\nD...   
7   [I-8604]  Source selection for Media output\nD...   
8   [I-26161]  FM Radio Stations switching\nDescri...   
9   [I-8800]  Receiving Call Notifications\nPrecon...   
10  [I-11467]  Revoke access to the vehicle from a...   
11  [I-8692]  Emergency Service Communication (ERA...   

                                       full_ssts_text  \
0   Make a call (B sample)\nFunctional Description...   
1   ERA Self-diagnosis\nFunctional Description\n\n...   
2   hotspot settings\nFunctional Description\nUser...   
3                                                None   
4   Mute or pause function\nFunctional Description...   
5   Favorite Song operation\nFunctional Descriptio...   
6   \nTurn on and off hotspot\nFunctional Descript...   
7   Switch music sources\nFunctional Description\n...   
8   Automatic search\nFunctional Description\nSupp...   
9   Receiving Call Notifications\nFunctional Descr...   
10  Users can remotely control the DK deletion thr...   
11  Manual dialing E-CALL\nFunctional Description\...   

                                                other  \
0                                                       
1   Description:\nERA self-diagnosis whether the s...   
2   Scope: \nSWP Actors:\nDriver \nOwner Requireme...   
3   Description: Requirenments:\nThe action is dis...   
4                                       Description:    
5   Description: Actors:\nCar User \nIVI Triggers:...   
6   Description: Scope:\nSWP Android \nVA \nSmartp...   
7   Description: Scope: This use case is about sel...   
8   Description: Requirenments:\nThe action is dis...   
9                                                       
10                                                      
11  Trigger:\nThe vehicle sensors detect a collisi...   

                                       postconditions  \
0   Postconditions:\nIf the call is initiated succ...   
1                                                       
2                                                       
3   Postconditions:\nout_2/out_5 displays the inte...   
4   Postconditions:\nThe user Mutes/Unmutes the FM...   
5   Postconditions:\nThe selected internet radio s...   
6                                                       
7   Postconditions:\nout_2/out_5 shows available m...   
8   Postconditions:\nout_5/out_2 shows the station...   
9   Postconditions:\nIf the call is accepted, the ...   
10  Postconditions:\nAccess has been successfully ...   
11  Postconditions：\nThe driver is able to communi...   

                                        main_scenario  \
0   Main Scenario:\nThe driver navigates to the 'C...   
1                                                       
2   Main Scenario:\nThe user guides

# Target

In [4]:
marked_data = pd.read_excel(data_dir / "raw_data" / "train_data_markup.xlsx")
marked_data = marked_data[["Number", "Complience Level"]]
marked_data = marked_data.rename(columns={"Number": "id", "Complience Level": "complience_level"})
marked_data["target"] = marked_data["complience_level"].fillna("NA").map(config["target_dict"])
marked_data = marked_data.drop("complience_level", axis=1)
marked_data["id"] = marked_data["id"].astype(str)

ds = ds.merge(marked_data, on="id")
ds

id                                          case_name  \
0    6583                 Driver initiate a call through SWP   
1   30371                                 ERA Self-diagnosis   
2   28561                    Setting Hotspot name & password   
3   26160              Filtering the internet radio stations   
4   25957                  Mute/unmute the FM Radio playback   
5   31523            Adding Internet Radio to Favorites List   
6   26771                     Turn on and off hotspot via VA   
7    8604                  Source selection for Media output   
8   26161                        FM Radio Stations switching   
9    8800                       Receiving Call Notifications   
10  11467  Revoke access to the vehicle from a driver or ...   
11   8692      Emergency Service Communication (ERA-Glonass)   

                                         full_uc_text  \
0   [I-6583]  Driver initiate a call through SWP\n...   
1   [I-30371]  ERA Self-diagnosis\nActors:\nERA-Gl...   
2   [I-28561]  Setting Hotspot name & password\nUs...   
3   [I-26160]  Filtering the internet radio statio...   
4   [I-25957]  Mute/unmute the FM Radio playback\n...   
5   [I-31523]  Adding Internet Radio to Favorites ...   
6   [I-26771]  Turn on and off hotspot via VA\n\nD...   
7   [I-8604]  Source selection for Media output\nD...   
8   [I-26161]  FM Radio Stations switching\nDescri...   
9   [I-8800]  Receiving Call Notifications\nPrecon...   
10  [I-11467]  Revoke access to the vehicle from a...   
11  [I-8692]  Emergency Service Communication (ERA...   

                                       full_ssts_text  \
0   Make a call (B sample)\nFunctional Description...   
1   ERA Self-diagnosis\nFunctional Description\n\n...   
2   hotspot settings\nFunctional Description\nUser...   
3                                                None   
4   Mute or pause function\nFunctional Description...   
5   Favorite Song operation\nFunctional Descriptio...   
6   \nTurn on and off hotspot\nFunctional Descript...   
7   Switch music sources\nFunctional Description\n...   
8   Automatic search\nFunctional Description\nSupp...   
9   Receiving Call Notifications\nFunctional Descr...   
10  Users can remotely control the DK deletion thr...   
11  Manual dialing E-CALL\nFunctional Description\...   

                                                other  \
0                                                       
1   Description:\nERA self-diagnosis whether the s...   
2   Scope: \nSWP Actors:\nDriver \nOwner Requireme...   
3   Description: Requirenments:\nThe action is dis...   
4                                       Description:    
5   Description: Actors:\nCar User \nIVI Triggers:...   
6   Description: Scope:\nSWP Android \nVA \nSmartp...   
7   Description: Scope: This use case is about sel...   
8   Description: Requirenments:\nThe action is dis...   
9                                                       
10                                                      
11  Trigger:\nThe vehicle sensors detect a collisi...   

                                       postconditions  \
0   Postconditions:\nIf the call is initiated succ...   
1                                                       
2                                                       
3   Postconditions:\nout_2/out_5 displays the inte...   
4   Postconditions:\nThe user Mutes/Unmutes the FM...   
5   Postconditions:\nThe selected internet radio s...   
6                                                       
7   Postconditions:\nout_2/out_5 shows available m...   
8   Postconditions:\nout_5/out_2 shows the station...   
9   Postconditions:\nIf the call is accepted, the ...   
10  Postconditions:\nAccess has been successfully ...   
11  Postconditions：\nThe driver is able to communi...   

                                        main_scenario  \
0   Main Scenario:\nThe driver navigates to the 'C...   
1                                                       
2   Main Scenario:\nThe user guides

# Свэговые графики

In [5]:
target_dict = config['target_dict']

number_to_abbr = {value: key for key, value in target_dict.items()}

In [6]:
target_percent = ds['target'].value_counts(normalize=True).sort_index() * 100
labels = target_percent.index.map(number_to_abbr)
print(labels)


Index(['NA', 'NC', 'PC', 'LC', 'FC'], dtype='object', name='target')


In [7]:
data = target_percent.reset_index()
data.columns = ['Target', 'Percentage']

fig = px.pie(
    data,
    names='Target',
    values='Percentage',
    title='Процентное распределение таргетов',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(template='plotly_white')

fig.show()

# Свэговые графики

In [5]:
target_dict = config['target_dict']

number_to_abbr = {value: key for key, value in target_dict.items()}

In [6]:
ds.to_csv(data_dir / "ds" / "ds.unl", sep="|", encoding="utf-8", index=False)

In [ ]:
plt.savefig('target_distribution_pie_chart.png', dpi=300, bbox_inches='tight')